<!-- # ============================================================
# 🔬 AI 기반 폐암 환자 맞춤형 신항원 발굴 및 암 백신 설계 시스템
# ============================================================
# 
# [프로젝트 목표]
# 폐암 환자의 유전자 변이 데이터를 분석하여, 면역세포가 암세포를 
# 정확히 인식할 수 있는 "신항원"을 AI로 발굴하고, 
# 환자 맞춤형 암 백신 설계 후보를 제시하는 시스템 구축
#
# [핵심 질문]
# 1. 폐암 환자의 암세포에서 어떤 단백질 조각(신항원)이 
#    면역세포의 공격 대상이 될 수 있는가?
# 2. AI가 46만 건의 데이터에서 학습한 패턴으로 
#    새로운 환자의 최적 백신 후보를 찾을 수 있는가?
#
# [왜 폐암인가?]
# - 국내 암 발생률 2위 (2023년 기준)
# - 5년 생존율 35.4% (조기 발견 시 치료 가능)
# - 유전자 변이가 다양해 맞춤 치료가 필수적
# ============================================================ -->

In [1]:
# [Cell 1] 라이브러리 임포트
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.patheffects as path_effects
from ydata_profiling import ProfileReport
import os


# 폰트 및 마이너스 깨짐 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


# 2. 데이터 로드
# Parquet 파일로 빠르게 로드 (CSV 대비 10배 이상 빠름)
file_path = 'dataset/Combined_Lung_Data.parquet' #데이터 변경해서사용
df = pd.read_parquet(file_path)

c:\code\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
c:\code\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 데이터의 첫 모습과 전체 컬럼 구조 확인
print("📂 전체 컬럼 개수:", len(df.columns))
# 컬럼찾기(컬럼명이 이상해서 대표 5개 데이터로 추가 확인)
print(df.head())

📂 전체 컬럼 개수: 14
     Type Tissue       Mutation         HLA    Peptide  Deep_bind  Deep_imm  \
0  Fusion   Lung     TOP1--CDH4  HLA-B41:01  IEADFRLNV     0.7018    0.8900   
1  Fusion   Lung  UNC13B--TAF7L  HLA-B07:02  RAHFPTGTV     0.9790    0.5627   
2  Fusion   Lung  ATP6V1E1--BID  HLA-C03:04  ITVNNGSSL     0.9974    0.9842   
3  Fusion   Lung  ATP6V1E1--BID  HLA-C03:04  RARDDLITV     0.8834    0.9886   
4  Fusion   Lung  ATP6V1E1--BID  HLA-C16:01  ITVNNGSSL     0.8356    0.9651   

   MHCf_rank (%)  Net4_aff (nM)  Net4_rank (%)  TPM Frequency in the tissue  \
0         0.1840          150.7         0.2149  NaN                   1/716   
1         0.8859          353.0         0.6820  NaN                   1/716   
2         0.3398           14.8         0.0600  NaN                   1/716   
3         0.3529          151.9         0.4723  NaN                   1/716   
4         0.3639           55.7         0.1848  NaN                   1/716   

  Frequency in all samples  source_

In [5]:
# [Cell 3 수정 예시]
# 'Tissue' 컬럼 등 폐암 정보가 들어있는 컬럼명을 확인 후 수정하세요
print("현재 데이터셋의 컬럼들:", df.columns.tolist())

# 2. 데이터가 어떻게 생겼는지 상위 3개만 출력
display(df.head(5))

현재 데이터셋의 컬럼들: ['Type', 'Tissue', 'Mutation', 'HLA', 'Peptide', 'Deep_bind', 'Deep_imm', 'MHCf_rank (%)', 'Net4_aff (nM)', 'Net4_rank (%)', 'TPM', 'Frequency in the tissue', 'Frequency in all samples', 'source_file']


,Type,Tissue,Mutation,HLA,Peptide,Deep_bind,Deep_imm,MHCf_rank (%),Net4_aff (nM),Net4_rank (%),TPM,Frequency in the tissue,Frequency in all samples,source_file
0,Fusion,Lung,TOP1--CDH4,HLA-B41:01,IEADFRLNV,0.7018,0.8900,0.1840,150.7,0.2149,NaN,1/716,1/4234,Fusion_Lung
1,Fusion,Lung,UNC13B--TAF7L,HLA-B07:02,RAHFPTGTV,0.9790,0.5627,0.8859,353.0,0.6820,NaN,1/716,1/4234,Fusion_Lung
2,Fusion,Lung,ATP6V1E1--BID,HLA-C03:04,ITVNNGSSL,0.9974,0.9842,0.3398,14.8,0.0600,NaN,1/716,1/4234,Fusion_Lung
3,Fusion,Lung,ATP6V1E1--BID,HLA-C03:04,RARDDLITV,0.8834,0.9886,0.3529,151.9,0.4723,NaN,1/716,1/4234,Fusion_Lung
4,Fusion,Lung,ATP6V1E1--BID,HLA-C16:01,ITVNNGSSL,0.8356,0.9651,0.3639,55.7,0.1848,NaN,1/716,1/4234,Fusion_Lung


In [ ]:
df_lung = df[is_lung_cancer].copy()


# [Cell 6 수정 예시]
label_col = 'Label' # 실제 라벨이 들어있는 컬럼명으로 수정
sequence_col = 'Peptide' # 아미노산 서열이 들어있는 컬럼명으로 수정

df_pos = df_lung[df_lung[label_col].str.contains('Positive', case=False, na=False)].copy()
df_neg = df_lung[df_lung[label_col].str.contains('Negative', case=False, na=False)].copy()

# 데이터 합치기 및 컬럼 정리
df_final = pd.concat([df_pos, df_neg])
df_final = df_final[[sequence_col, label_col]].copy()
df_final.columns = ['Sequence', 'Label']


In [ ]:
# ---------------------------------------------------------
# [단계 1] 확보된 폐암 데이터에서 필요한 칸(서열, 결과) 추출
# ---------------------------------------------------------# [수정된 Cell 6 로직]
# 1. 폐암 데이터 중 Positive와 Negative를 각각 추출
df_pos = df_lung[df_lung.iloc[:, 94].str.contains('Positive', case=False, na=False)].copy()
df_neg = df_lung[df_lung.iloc[:, 94].str.contains('Negative', case=False, na=False)].copy()

# 2. 데이터 불균형 해결 (Negative 데이터가 적다면 전체 데이터에서 정상 샘플을 더 가져와야 함)
# 여기서는 학습의 질을 위해 1:1 비율로 맞춤 샘플링을 진행합니다.
sample_size = min(len(df_pos), len(df_neg))

df_pos_sampled = df_pos.sample(n=sample_size, random_state=42)
df_neg_sampled = df_neg.sample(n=sample_size, random_state=42)

# 3. 두 데이터를 합쳐서 최종 학습셋 구성
df_final = pd.concat([df_pos_sampled, df_neg_sampled])
df_final = df_final.iloc[:, [11, 94]].copy() # 서열과 라벨만 추출
df_final.columns = ['Sequence', 'Label']

# 9글자 필터링
df_final = df_final[df_final['Sequence'].str.len() == 9].dropna()
print(f"✅ 전처리 완료! 최종 데이터 수: {len(df_final)}")
print(f"✅ 균형 잡힌 데이터 확보: Positive {sample_size}건 / Negative {sample_size}건")
print(df_final.head()) # 전처리된 5개 샘플 확인


✅ 전처리 완료! 최종 데이터 수: 10
✅ 균형 잡힌 데이터 확보: Positive 6건 / Negative 6건
       Sequence                  Label
6539  GLACHQLCA  Positive-Intermediate
5140  YMNGTMSQV          Positive-High
5132  LLMGTLGIV          Positive-High
6536  FLWGPRALV  Positive-Intermediate
6617  YLSGANLNL           Positive-Low


In [ ]:
df_report = df_final.copy()

# 1. 서열의 길이 (모두 9겠지만 통계에는 잡힙니다)
df_report['Seq_Length'] = df_report['Sequence'].str.len()

# 2. 특정 아미노산의 포함 여부 (예: 암세포와 관련 깊은 'L', 'V' 등)
df_report['Has_Leucine(L)'] = df_report['Sequence'].str.contains('L').astype(int)
df_report['Has_Valine(V)'] = df_report['Sequence'].str.contains('V').astype(int)

# 3. 서열의 첫 번째와 마지막 아미노산 (위치별 특성 파악용)
df_report['First_AA'] = df_report['Sequence'].str[0]
df_report['Last_AA'] = df_report['Sequence'].str[-1]

# 4. 아미노산의 소수성(Hydrophobicity) 대략적 분류 (간단 예시)
# 서열에 특정 문자가 많으면 1, 아니면 0 식으로 특징 추출
df_report['Hydrophobic_Score'] = df_report['Sequence'].apply(lambda x: sum(1 for aa in x if aa in 'AILMFPWV'))

# 이제 변수가 늘어난 데이터로 리포트 생성!
from ydata_profiling import ProfileReport
profile = ProfileReport(df_report, title="폐암 신항원 정밀 분석 리포트")
profile.to_file("lung_cancer_rich_report.html")

print("-" * 50)
print(f"✅ 최종 9-mer 폐암 데이터 확보: {len(df_final)}건")
print("-" * 50)
print(df_final.head())

Export report to file: 100%|██████████| 1/1 [00:00<?, ?it/s]

--------------------------------------------------
✅ 최종 9-mer 폐암 데이터 확보: 10건
--------------------------------------------------
       Sequence                  Label
6539  GLACHQLCA  Positive-Intermediate
5140  YMNGTMSQV          Positive-High
5132  LLMGTLGIV          Positive-High
6536  FLWGPRALV  Positive-Intermediate
6617  YLSGANLNL           Positive-Low


In [ ]:
# 1. 아미노산 빈도 히트맵
def draw_heatmap(df):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    heatmap_data = np.zeros((20, 9))
    for seq in df['Sequence']:
        for i, aa in enumerate(seq):
            if aa in amino_acids:
                heatmap_data[amino_acids.index(aa), i] += 1
    plt.figure(figsize=(10, 7))
    sns.heatmap(heatmap_data, xticklabels=range(1, 10), yticklabels=list(amino_acids), cmap='YlGnBu')
    plt.title('신항원 서열 위치별 아미노산 분포 (Heatmap)')
    plt.show()

# 2. 정답 레이블 분포 (히스토그램/바 차트)
def draw_label_dist(df):
    plt.figure(figsize=(6, 4))
    sns.countplot(x='Label', data=df)
    plt.title('면역 반응 정답(Label) 분포')
    plt.show()

# 실행
draw_heatmap(df_final)
draw_label_dist(df_final)

C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\1817969079.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\1817969079.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
# 1. 아미노산 인코딩 함수 (노트북 분석 결과 반영)
def encode_sequence(seq):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY' # 20개 표준 순서
    aa_to_int = {aa: i for i, aa in enumerate(amino_acids)}
    matrix = np.zeros((9, 20))
    for i, aa in enumerate(seq.upper()):
        if aa in aa_to_int:
            matrix[i, aa_to_int[aa]] = 1
    return matrix

# 1-2. 리스트 전체 변환 함수
def encode_all_sequences(sequences):
    encoded_list = [encode_sequence(s) for s in sequences]
    return np.array(encoded_list)

# 2. 모델 생성 함수 정의
def build_final_model():
    model = models.Sequential([
        layers.Input(shape=(9, 20)),
        
        # 1. 첫 번째 컨볼루션 층 (패턴 추출)
        layers.Conv1D(64, kernel_size=3, padding='same', activation='relu'),
        layers.BatchNormalization(), # 학습을 안정화시키는 최적화 도구
        
        # 2. 두 번째 컨볼루션 층 (더 복잡한 패턴 추출)
        layers.Conv1D(128, kernel_size=2, padding='same', activation='relu'),
        layers.GlobalMaxPooling1D(), # 가장 중요한 특징만 뽑아냄
        
        # 3. 전결합 층 및 과적합 방지
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3), # 데이터가 적을 때 모델이 외우기만 하는 현상 방지
        
        layers.Dense(1, activation='sigmoid') # 신항원일 확률 (0~1) 출력
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 3. 데이터 준비 및 모델 생성 실행
# X, y 데이터 생성
X = encode_all_sequences(df_final['Sequence'].values)
y = (df_final['Label'].str.contains('Positive', case=False)).astype(int).values
# 데이터셋 분리 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
model = build_final_model()

print(f"✅ 학습 데이터 준비 완료: {X_train.shape}")
print(f"✅ 신항원(Positive) 샘플 비중: {y.mean()*100:.2f}%")

✅ 학습 데이터 준비 완료: (8, 9, 20)
✅ 신항원(Positive) 샘플 비중: 50.00%


In [ ]:
# ---------------------------------------------------------
# ✅ 학습 진행
# ---------------------------------------------------------
# 모델 설정 (학습 지표 추가)
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# 3. 모델 학습 시작 
print("🚀 개선된 AI 모델로 학습을 시작합니다...")

# 실제 신항원을 더 잘 맞히기 위해 가중치(class_weight)를 추가하는 것이 좋습니다.
# Positive(1) 데이터를 더 엄격하게 배우도록 2.5배 가중치를 둡니다.
history = model.fit(X_train, y_train, 
                    epochs=20,           # 학습 횟수를 조금 더 늘려 숙련도를 높입니다.
                    batch_size=32, 
                    validation_data=(X_test, y_test),
                    class_weight={0: 1.0, 1: 2.5}) # 실제 항원을 놓치지 않게 강조



🚀 개선된 AI 모델로 학습을 시작합니다...
Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7500 - loss: 0.9950 - precision: 0.6667 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6818 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.8750 - loss: 0.7054 - precision: 1.0000 - recall: 0.7500 - val_accuracy: 1.0000 - val_loss: 0.6838 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6250 - loss: 0.6917 - precision: 0.5714 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6848 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.6250 - loss: 0.7898 - precision: 0.5714 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6833 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.8750 - loss: 0.3649 - precision: 0.8000 - recall: 1.0000 - val_accuracy: 0.50

In [ ]:
# 1. 모델 평가 (지표가 여러 개이므로 리스트로 받습니다)
evaluation_results = model.evaluate(X_test, y_test)

# 모델 컴파일 시 설정한 순서대로 값이 들어옵니다: [Loss, Accuracy, Precision, Recall]
loss = evaluation_results[0]
accuracy = evaluation_results[1]
precision = evaluation_results[2]
recall = evaluation_results[3]

print(f"📊 최종 테스트 손실(Loss): {loss:.4f}")
print(f"🎯 최종 테스트 정확도(Accuracy): {accuracy*100:.2f}%")
print(f"⚖️ 정밀도(Precision): {precision*100:.2f}%  <- '암'이라고 한 것 중 진짜 암인 비율")
print(f"📈 재현율(Recall): {recall*100:.2f}%  <- 실제 암 서열을 놓치지 않고 찾아낸 비율")

# 2. 샘플 데이터 예측 수행 (이 부분은 동일합니다)
samples = X_test[:5]
predictions = model.predict(samples)

print("\n--- 🔬 신항원 예측 수행 결과 (샘플) ---")

# predictions에 들어있는 실제 개수만큼만 반복합니다.
# 만약 5개보다 많을 때 딱 5개만 보고 싶다면 min(len(predictions), 5)를 쓰세요.
num_samples = len(predictions) 

for i in range(num_samples):
    # predictions[i]가 배열 형태인 경우와 단일 값인 경우를 모두 고려
    prob = predictions[i][0] if hasattr(predictions[i], "__len__") else predictions[i]
    
    result = "Positive" if prob > 0.5 else "Negative"
    probability = prob * 100
    print(f"서열 샘플 {i+1}번 -> 예측 결과: {result} (확률: {probability:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5000 - loss: 0.6716 - precision: 0.5000 - recall: 1.0000
📊 최종 테스트 손실(Loss): 0.6716
🎯 최종 테스트 정확도(Accuracy): 50.00%
⚖️ 정밀도(Precision): 50.00%  <- '암'이라고 한 것 중 진짜 암인 비율
📈 재현율(Recall): 100.00%  <- 실제 암 서열을 놓치지 않고 찾아낸 비율
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step

--- 🔬 신항원 예측 수행 결과 (샘플) ---
서열 샘플 1번 -> 예측 결과: Positive (확률: 52.09%)
서열 샘플 2번 -> 예측 결과: Positive (확률: 54.48%)


In [ ]:
from ydata_profiling import ProfileReport

# 1. 리포트 생성 (반드시 df_report를 넣어야 8개 변수가 보입니다!)
profile = ProfileReport(df_report, 
                        title="폐암 신항원 정밀 분석 리포트 (8개 변수)", 
                        explorative=True,
                        minimal=False)

# 2. HTML 파일로 저장
profile.to_file("lung_cancer_FINAL_report.html")

# 3. 완료 메시지 (파일 이름 일치시키기)
print("✨ 데이터 분석 리포트가 'lung_cancer_FINAL_report.html'로 저장되었습니다!")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 648.57it/s]

✨ 데이터 분석 리포트가 'lung_cancer_FINAL_report.html'로 저장되었습니다!


In [ ]:
# 기존 .h5 대신 최신 표준인 .keras를 사용합니다.
model.save("lung_cancer_model.keras")

print("✨ 모델이 'lung_cancer_model.keras'로 안전하게 저장되었습니다!")

✨ 모델이 'lung_cancer_model.keras'로 안전하게 저장되었습니다!


In [ ]:

print(f"TensorFlow 버전: {tf.__version__}")


# [Cell 2] 환경 설정
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style("whitegrid")

print("✅ 시각화 환경 설정 완료")


TensorFlow 버전: 2.20.0
✅ 시각화 환경 설정 완료


In [ ]:
# [Cell 3] 데이터 로드 (최적화된 방식)
print("\n" + "="*70)
print("📂 STEP 1: 데이터 로드 (46만 건의 MHC-펩타이드 상호작용 데이터)")
print("="*70)
print(f"✅ 전체 데이터 로드 완료: {len(df):,}건")
print(f"✅ 컬럼 수: {len(df.columns)}개")



📂 STEP 1: 데이터 로드 (46만 건의 MHC-펩타이드 상호작용 데이터)
✅ 전체 데이터 로드 완료: 100,000건
✅ 컬럼 수: 112개


In [ ]:
# [Cell 4] 폐암 데이터 필터링
print("\n" + "="*70)
print("🔍 STEP 2: 폐암 관련 데이터 추출")
print("="*70)

# 8번째 컬럼에서 폐암 키워드 검색
is_lung_cancer = df.iloc[:, 8].str.contains(
    'Lung|Adenocarcinoma|NSCLC|Cancer', 
    case=False, 
    na=False
)

df_lung = df[is_lung_cancer].copy()

print(f"✅ 폐암 관련 데이터 추출: {len(df_lung):,}건")
print(f"   전체 대비 비율: {len(df_lung)/len(df)*100:.2f}%")


🔍 STEP 2: 폐암 관련 데이터 추출
✅ 폐암 관련 데이터 추출: 152건
   전체 대비 비율: 0.15%


In [ ]:
# [Cell 5] 신항원 후보 데이터 정제
print("\n" + "="*70)
print("🧬 STEP 3: 신항원 후보 서열 정제 (9-mer 펩타이드)")
print("="*70)

# 필요한 컬럼만 추출: [서열, 면역반응 결과]
df_final = df_lung.iloc[:, [11, 94]].copy()
df_final.columns = ['Sequence', 'Label']

# 9글자 서열만 필터링 (MHC Class I에 최적화된 길이)
df_final['Sequence'] = df_final['Sequence'].astype(str)
df_final = df_final[df_final['Sequence'].str.len() == 9].dropna()

# ⚠️ CRITICAL: Label 확인 및 정제
print("\n[원본 Label 값 확인]")
print(df_final['Label'].value_counts())



🧬 STEP 3: 신항원 후보 서열 정제 (9-mer 펩타이드)

[원본 Label 값 확인]
Label
Positive-Intermediate    47
Positive-High            37
Positive-Low             11
Positive                  6
Negative                  5
Name: count, dtype: int64


In [ ]:
# Label 정제 (Positive/Negative로 단순화)
df_final['Label_Clean'] = df_final['Label'].apply(
    lambda x: 'Positive' if 'Positive' in str(x) else 'Negative'
)

print(f"\n✅ 9-mer 펩타이드 데이터: {len(df_final):,}건")
print(f"\n[정제된 Label 분포]")
print(df_final['Label_Clean'].value_counts())
print(f"\nPositive 비율: {(df_final['Label_Clean'] == 'Positive').sum() / len(df_final) * 100:.2f}%")



✅ 9-mer 펩타이드 데이터: 106건

[정제된 Label 분포]
Label_Clean
Positive    101
Negative      5
Name: count, dtype: int64

Positive 비율: 95.28%


In [ ]:
# ⚠️ 데이터 불균형 심각하면 경고
pos_ratio = (df_final['Label_Clean'] == 'Positive').sum() / len(df_final)
if pos_ratio > 0.95:
    print("\n⚠️ 경고: Positive 비율이 95% 이상입니다!")
    print("   데이터가 심하게 불균형합니다. 모델 성능에 문제가 있을 수 있습니다.")
    print("   → 해결책: SMOTE, Under-sampling, Class Weight 조정 필요")

# 샘플 데이터 확인
print("\n[데이터 샘플 - Positive 5개]")
print(df_final[df_final['Label_Clean'] == 'Positive'][['Sequence', 'Label_Clean']].head())

print("\n[데이터 샘플 - Negative 5개]")
negative_samples = df_final[df_final['Label_Clean'] == 'Negative'][['Sequence', 'Label_Clean']].head()
if len(negative_samples) > 0:
    print(negative_samples)
else:
    print("⚠️ Negative 샘플이 없습니다! 이는 심각한 문제입니다.")



⚠️ 경고: Positive 비율이 95% 이상입니다!
   데이터가 심하게 불균형합니다. 모델 성능에 문제가 있을 수 있습니다.
   → 해결책: SMOTE, Under-sampling, Class Weight 조정 필요

[데이터 샘플 - Positive 5개]
      Sequence Label_Clean
732  KLLMVLMLA    Positive
733  LMVLMLAAL    Positive
734  FLNQTDETL    Positive
735  TLSNVEVFM    Positive
736  MQLIYDSSL    Positive

[데이터 샘플 - Negative 5개]
       Sequence Label_Clean
6521  ALCRWGLLL    Negative
6524  ALCRWGLLL    Negative
6548  ILHNGAYSL    Negative
6577  LLTFWNPPT    Negative
6618  YLSGANLNL    Negative


In [ ]:
# [Cell 6] 추가 변수 생성 (EDA 및 리포트용)
print("\n" + "="*70)
print("🔧 STEP 4: 추가 변수 생성 (분석 강화)")
print("="*70)

df_analysis = df_final.copy()
df_analysis['Seq_Length'] = df_analysis['Sequence'].str.len()
df_analysis['Has_Leucine'] = df_analysis['Sequence'].str.contains('L').astype(int)
df_analysis['Has_Valine'] = df_analysis['Sequence'].str.contains('V').astype(int)
df_analysis['First_AA'] = df_analysis['Sequence'].str[0]
df_analysis['Last_AA'] = df_analysis['Sequence'].str[-1]
df_analysis['Hydrophobic_Count'] = df_analysis['Sequence'].apply(
    lambda x: sum(1 for aa in x if aa in 'AILMFPWV')
)

print("✅ 추가 변수 6개 생성 완료")
print(f"   총 변수 개수: {len(df_analysis.columns)}개")


🔧 STEP 4: 추가 변수 생성 (분석 강화)
✅ 추가 변수 6개 생성 완료
   총 변수 개수: 9개


In [ ]:
# [Cell 7] 화려한 데이터 시각화 (발표용)
print("\n" + "="*70)
print("🎨 STEP 5: 데이터 시각화 (화려한 버전)")
print("="*70)

fig = plt.figure(figsize=(18, 12))
fig.patch.set_facecolor('#f8f9fa')

# 1-1. 화려한 Label 분포 (3D 효과)
ax1 = plt.subplot(2, 3, 1)
ax1.set_facecolor('#ffffff')

label_counts = df_analysis['Label_Clean'].value_counts()
colors_gradient = ['#ff6b6b', '#4ecdc4']

bars = ax1.bar(range(len(label_counts)), label_counts.values, 
               color=colors_gradient, alpha=0.85, edgecolor='black', linewidth=2)



🎨 STEP 5: 데이터 시각화 (화려한 버전)


In [ ]:
# 3D 효과 추가
for i, bar in enumerate(bars):
    height = bar.get_height()
    shadow = Rectangle((bar.get_x() + 0.02, 0.02), bar.get_width(), height,
                       facecolor='gray', alpha=0.3, zorder=0)
    ax1.add_patch(shadow)
    
    text = ax1.text(bar.get_x() + bar.get_width()/2, height + 5000,
                   f'{label_counts.values[i]:,}개\n({label_counts.values[i]/len(df_final)*100:.1f}%)',
                   ha='center', va='bottom', fontsize=14, fontweight='bold',
                   color=colors_gradient[i])
    text.set_path_effects([path_effects.withStroke(linewidth=3, foreground='white')])

ax1.set_xticks(range(len(label_counts)))
ax1.set_xticklabels(label_counts.index, fontsize=13, fontweight='bold')
ax1.set_ylabel('데이터 개수', fontsize=13, fontweight='bold')
ax1.set_title('💉 면역 반응 데이터 분포', fontsize=16, fontweight='bold', 
              pad=20, color='#2c3e50')
ax1.grid(axis='y', alpha=0.3, linestyle='--', linewidth=1.5)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)


In [ ]:
# 1-2. 도넛 차트
ax2 = plt.subplot(2, 3, 2)
ax2.set_facecolor('#ffffff')

explode = (0.05, 0.05)
colors_pie = ['#ff6b6b', '#4ecdc4']
wedges, texts, autotexts = ax2.pie(label_counts.values, 
                                     explode=explode,
                                     labels=label_counts.index,
                                     autopct='%1.1f%%',
                                     colors=colors_pie,
                                     startangle=90,
                                     textprops={'fontsize': 13, 'fontweight': 'bold'},
                                     shadow=True,
                                     wedgeprops={'edgecolor': 'white', 'linewidth': 3})

centre_circle = plt.Circle((0, 0), 0.60, fc='white', linewidth=3, edgecolor='#2c3e50')
ax2.add_artist(centre_circle)
ax2.text(0, 0, f'{len(df_final):,}건', ha='center', va='center',
         fontsize=20, fontweight='bold', color='#2c3e50')
ax2.set_title('📊 데이터 구성 비율', fontsize=16, fontweight='bold', 
              pad=20, color='#2c3e50')


Text(0.5, 1.0, '📊 데이터 구성 비율')

In [ ]:
# 1-3. 그라데이션 히트맵
ax3 = plt.subplot(2, 3, 3)
ax3.set_facecolor('#ffffff')

amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
heatmap_data = np.zeros((20, 9))

for seq in df_final['Sequence'][:20000]:
    for i, aa in enumerate(seq):
        if aa in amino_acids:
            heatmap_data[amino_acids.index(aa), i] += 1

colors_heat = ['#08519c', '#3182bd', '#6baed6', '#bdd7e7', '#eff3ff',
               '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
cmap = LinearSegmentedColormap.from_list('custom', colors_heat, N=100)

im = ax3.imshow(heatmap_data, cmap=cmap, aspect='auto', interpolation='bilinear')

ax3.set_xticks(range(9))
ax3.set_xticklabels(range(1, 10), fontsize=12, fontweight='bold')
ax3.set_yticks(range(20))
ax3.set_yticklabels(list(amino_acids), fontsize=11, fontweight='bold')
ax3.set_xlabel('펩타이드 위치', fontsize=13, fontweight='bold')
ax3.set_ylabel('아미노산', fontsize=13, fontweight='bold')
ax3.set_title('🧬 신항원 서열 패턴 히트맵', fontsize=16, fontweight='bold',
              pad=20, color='#2c3e50')

cbar = plt.colorbar(im, ax=ax3, fraction=0.046, pad=0.04)
cbar.set_label('출현 빈도', fontsize=12, fontweight='bold')

In [ ]:
# 2-1. 그라데이션 히스토그램
ax4 = plt.subplot(2, 3, 4)
ax4.set_facecolor('#ffffff')

hydro_counts = df_analysis['Hydrophobic_Count']
n, bins, patches = ax4.hist(hydro_counts, bins=10, edgecolor='white', linewidth=2)

cm = plt.cm.get_cmap('RdYlGn_r')
norm = plt.Normalize(vmin=min(n), vmax=max(n))

for i, patch in enumerate(patches):
    color = cm(norm(n[i]))
    patch.set_facecolor(color)
    patch.set_alpha(0.85)

ax4.axvline(hydro_counts.mean(), color='red', linestyle='--', linewidth=3,
           label=f'평균: {hydro_counts.mean():.1f}', alpha=0.8)

ax4.set_xlabel('소수성 아미노산 개수', fontsize=13, fontweight='bold')
ax4.set_ylabel('빈도', fontsize=13, fontweight='bold')
ax4.set_title('💧 서열 소수성 분포', fontsize=16, fontweight='bold',
              pad=20, color='#2c3e50')
ax4.legend(fontsize=12, frameon=True, shadow=True)
ax4.grid(axis='y', alpha=0.3, linestyle='--')
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)

C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\2666653135.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cm = plt.cm.get_cmap('RdYlGn_r')


In [ ]:
# 2-2. 화려한 수평 막대
ax5 = plt.subplot(2, 3, 5)
ax5.set_facecolor('#ffffff')

first_aa_counts = df_analysis['First_AA'].value_counts().head(10)
colors_bar = plt.cm.plasma(np.linspace(0.2, 0.9, len(first_aa_counts)))

bars = ax5.barh(range(len(first_aa_counts)), first_aa_counts.values,
                color=colors_bar, edgecolor='black', linewidth=1.5, alpha=0.85)

for i, (bar, val) in enumerate(zip(bars, first_aa_counts.values)):
    ax5.text(val + 1000, i, f'{val:,}', va='center', fontsize=11, fontweight='bold')

ax5.set_yticks(range(len(first_aa_counts)))
ax5.set_yticklabels(first_aa_counts.index, fontsize=13, fontweight='bold')
ax5.set_xlabel('출현 빈도', fontsize=13, fontweight='bold')
ax5.set_title('🔤 첫 번째 위치 아미노산 Top 10', fontsize=16, fontweight='bold',
              pad=20, color='#2c3e50')
ax5.invert_yaxis()
ax5.grid(axis='x', alpha=0.3, linestyle='--')
ax5.spines['top'].set_visible(False)
ax5.spines['right'].set_visible(False)

In [ ]:
# 2-3. 박스플롯
ax6 = plt.subplot(2, 3, 6)
ax6.set_facecolor('#ffffff')

positive_data = df_analysis[df_analysis['Label_Clean'] == 'Positive']['Hydrophobic_Count']
negative_data = df_analysis[df_analysis['Label_Clean'] == 'Negative']['Hydrophobic_Count']

data_box = [positive_data, negative_data]
labels_box = ['Positive', 'Negative']

bp = ax6.boxplot(data_box, labels=labels_box, patch_artist=True,
                 showmeans=True, meanline=True,
                 boxprops=dict(linewidth=2, edgecolor='black'),
                 whiskerprops=dict(linewidth=2, color='black'),
                 capprops=dict(linewidth=2, color='black'),
                 medianprops=dict(linewidth=3, color='red'),
                 meanprops=dict(linewidth=3, color='blue', linestyle='--'))

box_colors = ['#ff6b6b', '#4ecdc4']
for patch, color in zip(bp['boxes'], box_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax6.set_ylabel('소수성 아미노산 개수', fontsize=13, fontweight='bold')
ax6.set_xticklabels(labels_box, fontsize=13, fontweight='bold')
ax6.set_title('📦 Label별 소수성 비교', fontsize=16, fontweight='bold',
              pad=20, color='#2c3e50')
ax6.grid(axis='y', alpha=0.3, linestyle='--')
ax6.spines['top'].set_visible(False)
ax6.spines['right'].set_visible(False)

fig.suptitle('🔬 폐암 신항원 데이터 정밀 분석 대시보드', 
             fontsize=22, fontweight='bold', y=0.98, color='#2c3e50')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('eda_analysis_final.png', dpi=300, bbox_inches='tight', 
            facecolor='#f8f9fa', edgecolor='none')
print("✅ 시각화 저장: eda_analysis_final.png")
plt.show()


C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\3576570574.py:11: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax6.boxplot(data_box, labels=labels_box, patch_artist=True,
C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\3576570574.py:35: UserWarning: Glyph 45936 (\N{HANGUL SYLLABLE DE}) missing from font(s) Arial.
  plt.tight_layout(rect=[0, 0, 1, 0.96])
C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\3576570574.py:35: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from font(s) Arial.
  plt.tight_layout(rect=[0, 0, 1, 0.96])
C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\3576570574.py:35: UserWarning: Glyph 53552 (\N{HANGUL SYLLABLE TEO}) missing from font(s) Arial.
  plt.tight_layout(rect=[0, 0, 1, 0.96])
C:\Users\human-24\AppData\Local\Temp\ipykernel_2900\3576570574.py:35: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missin

MemoryError: bad allocation

In [ ]:
# [Cell 8] YData Profiling 리포트 생성
print("\n" + "="*70)
print("📊 STEP 6: 상세 데이터 분석 리포트 생성 (YData Profiling)")
print("="*70)

from ydata_profiling import ProfileReport

profile = ProfileReport(df_analysis, 
                        title="폐암 신항원 정밀 분석 리포트", 
                        explorative=True,
                        minimal=False)

profile.to_file("lung_cancer_FINAL_report.html")

print("✅ 데이터 분석 리포트가 'lung_cancer_FINAL_report.html'로 저장되었습니다!")


In [ ]:
# [Cell 9] 데이터 전처리 (원핫 인코딩)
print("\n" + "="*70)
print("🔧 STEP 7: 데이터 전처리 (원핫 인코딩)")
print("="*70)

def neoantigen_onehot(sequences):
    """아미노산 서열을 딥러닝 입력용 숫자 행렬로 변환"""
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    aa_to_int = {aa: i for i, aa in enumerate(amino_acids)}
    
    encoded = []
    for seq in sequences:
        matrix = np.zeros((9, 20))
        for i, aa in enumerate(seq):
            if aa in aa_to_int:
                matrix[i, aa_to_int[aa]] = 1
        encoded.append(matrix)
    return np.array(encoded)

X = neoantigen_onehot(df_final['Sequence'].values)
y = (df_final['Label_Clean'] == 'Positive').astype(int).values

print(f"✅ X (입력 데이터) shape: {X.shape}")
print(f"   - {X.shape[0]:,}개 샘플")
print(f"   - 각 샘플: 9개 위치 × 20종 아미노산")
print(f"\n✅ y (정답) shape: {y.shape}")
print(f"   - Positive (1): {y.sum():,}개 ({y.sum()/len(y)*100:.1f}%)")
print(f"   - Negative (0): {len(y)-y.sum():,}개 ({(len(y)-y.sum())/len(y)*100:.1f}%)")


In [ ]:
# ⚠️ 클래스 불균형 심각하면 경고
if y.sum()/len(y) > 0.95:
    print("\n" + "🚨"*35)
    print("⚠️⚠️⚠️ 심각한 클래스 불균형 감지! ⚠️⚠️⚠️")
    print("🚨"*35)
    print(f"Positive 비율: {y.sum()/len(y)*100:.2f}%")
    print("이 상태로 학습하면 모델이 무조건 Positive로 예측합니다!")
    print("\n해결책을 선택하세요:")
    print("1. Under-sampling: Positive 데이터 줄이기")
    print("2. SMOTE: Negative 데이터 합성")
    print("3. Class Weight: 손실 함수에 가중치 적용")
    print("="*70)
    
    # 자동으로 Class Weight 계산
    from sklearn.utils.class_weight import compute_class_weight
    class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(f"\n📊 권장 Class Weight:")
    print(f"   Negative (0): {class_weights[0]:.2f}")
    print(f"   Positive (1): {class_weights[1]:.2f}")
    print("\n→ 모델 학습 시 이 가중치를 적용하겠습니다.")


In [ ]:
# [Cell 10] 데이터 분할
print("\n" + "="*70)
print("✂️ STEP 8: 학습/테스트 데이터 분할 (80:20)")
print("="*70)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ 학습 데이터: {len(X_train):,}개 ({len(X_train)/len(X)*100:.1f}%)")
print(f"✅ 테스트 데이터: {len(X_test):,}개 ({len(X_test)/len(X)*100:.1f}%)")
print(f"\n[클래스 균형 확인]")
print(f"학습 데이터 Positive 비율: {y_train.sum()/len(y_train)*100:.2f}%")
print(f"테스트 데이터 Positive 비율: {y_test.sum()/len(y_test)*100:.2f}%")


In [ ]:
# [Cell 11] 1D-CNN 모델 구축
print("\n" + "="*70)
print("🤖 STEP 9: 딥러닝 모델 구축 (1D-CNN)")
print("="*70)

model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(9, 20)),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.2),
    layers.Conv1D(32, kernel_size=2, activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)


# [추가] 1. 클래스 가중치 자동 계산 (불균형 해소 핵심)
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(weights))

# [추가] 2. 과적합 방지를 위한 EarlyStopping
# dl2 파일의 학습 안정성 로직을 참고하여 추가
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)

# [수정] 3. 모델 학습 부분
history = model.fit(
    X_train, y_train,
    epochs=100,             # 에포크를 늘려도 early_stopping이 있어 안전함
    batch_size=64,          # dl2 스타일의 배치 사이즈 조정
    validation_split=0.2,
    class_weight=class_weights, # 가중치 적용!
    callbacks=[early_stopping],  # 검증 손실이 안 줄어들면 자동 멈춤
    verbose=1
)

print(model.summary())


In [ ]:
# [Cell 12] 모델 학습
print("\n" + "="*70)
print("🚀 STEP 10: 모델 학습 시작")
print("="*70)

# Class Weight 적용 여부 확인
if y.sum()/len(y) > 0.95:
    print("⚠️ 클래스 불균형으로 인해 Class Weight를 적용합니다.")
    history = model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=32,
        validation_data=(X_test, y_test),
        class_weight=class_weight_dict,
        verbose=1
    )
else:
    history = model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1
    )

print("\n✅ 학습 완료!")

In [ ]:
# [Cell 13] 학습 결과 시각화
print("\n" + "="*70)
print("📈 STEP 11: 학습 결과 시각화")
print("="*70)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.patch.set_facecolor('#f8f9fa')

ax1 = axes[0]
ax1.plot(history.history['accuracy'], label='학습 정확도', linewidth=2, marker='o')
ax1.plot(history.history['val_accuracy'], label='검증 정확도', linewidth=2, marker='s')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('1️⃣ 모델 정확도', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
final_acc = history.history['val_accuracy'][-1]
ax1.text(9, final_acc, f'{final_acc*100:.2f}%', 
         fontsize=12, fontweight='bold', color='red')

ax2 = axes[1]
ax2.plot(history.history['loss'], label='학습 손실', linewidth=2, marker='o')
ax2.plot(history.history['val_loss'], label='검증 손실', linewidth=2, marker='s')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.set_title('2️⃣ 모델 손실', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

ax3 = axes[2]
ax3.plot(history.history['auc'], label='학습 AUC', linewidth=2, marker='o')
ax3.plot(history.history['val_auc'], label='검증 AUC', linewidth=2, marker='s')
ax3.set_xlabel('Epoch', fontsize=12)
ax3.set_ylabel('AUC', fontsize=12)
ax3.set_title('3️⃣ AUC Score', fontsize=14, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(True, alpha=0.3)
final_auc = history.history['val_auc'][-1]
ax3.text(9, final_auc, f'{final_auc:.4f}', 
         fontsize=12, fontweight='bold', color='red')

plt.tight_layout()
plt.savefig('training_results.png', dpi=300, bbox_inches='tight', facecolor='#f8f9fa')
print("✅ 시각화 저장: training_results.png")
plt.show()

In [ ]:
# [Cell 14] 모델 평가
print("\n" + "="*70)
print("🎯 STEP 12: 최종 모델 성능 평가")
print("="*70)

test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=0)

print(f"""
╔════════════════════════════════════════════╗
║         최종 모델 성능 지표                 ║
╠════════════════════════════════════════════╣
║  정확도 (Accuracy):  {test_acc*100:>6.2f}%          ║
║  AUC Score:          {test_auc:>6.4f}            ║
║  손실 (Loss):        {test_loss:>6.4f}            ║
╚════════════════════════════════════════════╝
""")

y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.patch.set_facecolor('#f8f9fa')

ax1 = axes[0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'],
            annot_kws={'size': 14, 'weight': 'bold'})
ax1.set_xlabel('예측값', fontsize=12, fontweight='bold')
ax1.set_ylabel('실제값', fontsize=12, fontweight='bold')
ax1.set_title('Confusion Matrix', fontsize=14, fontweight='bold')

ax2 = axes[1]
y_pred_proba = model.predict(X_test, verbose=0)
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

ax2.plot(fpr, tpr, color='darkorange', lw=3, label=f'ROC curve (AUC = {roc_auc:.4f})')
ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
ax2.set_xlabel('False Positive Rate', fontsize=12)
ax2.set_ylabel('True Positive Rate', fontsize=12)
ax2.set_title('ROC Curve', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('evaluation_results.png', dpi=300, bbox_inches='tight', facecolor='#f8f9fa')
print("✅ 시각화 저장: evaluation_results.png")
plt.show()

print("\n[상세 분류 리포트]")
print(classification_report(y_test, y_pred, 
                           target_names=['Negative', 'Positive'],
                           digits=4))


In [ ]:
# [Cell 15] 모델 저장
print("\n" + "="*70)
print("💾 STEP 13: 모델 저장")
print("="*70)

model.save("lung_cancer_model.keras")
print("✅ 모델이 'lung_cancer_model.keras'로 저장되었습니다!")
print("   → Streamlit 앱에서 이 파일을 불러와 실시간 예측에 사용합니다.")

# [Cell 16] 샘플 예측 데모
print("\n" + "="*70)
print("🔬 STEP 14: 신항원 예측 데모 (샘플 5개)")
print("="*70)

samples = X_test[:5]
sample_labels = y_test[:5]
predictions = model.predict(samples, verbose=0)

print("\n실제 환자 데이터를 시뮬레이션한 예측 결과:\n")
for i in range(5):
    actual = "Positive" if sample_labels[i] == 1 else "Negative"
    pred_prob = predictions[i][0]
    predicted = "Positive" if pred_prob > 0.5 else "Negative"
    
    status = "✅" if actual == predicted else "❌"
    
    print(f"{status} 샘플 {i+1}")
    print(f"   실제값: {actual}")
    print(f"   예측값: {predicted} (확률: {pred_prob*100:.2f}%)")
    print()

In [ ]:
# [Cell 17] 프로젝트 최종 요약
print("\n" + "="*70)
print("📊 프로젝트 최종 요약")
print("="*70)

print(f"""
🎯 [달성한 목표]
✅ 46만 건의 MHC-펩타이드 데이터에서 폐암 관련 신항원 데이터 추출
✅ 1D-CNN 딥러닝 모델로 {test_acc*100:.2f}% 정확도 달성
✅ 환자 맞춤형 암 백신 후보 예측 시스템 구축

📈 [핵심 성과]
• 정확도: {test_acc*100:.2f}%
• AUC Score: {test_auc:.4f}
• 처리 데이터: {len(df_final):,}건의 9-mer 펩타이드

🔬 [실제 활용 방안]
1. 환자의 유전자 검사 결과 → 돌연변이 펩타이드 추출
2. AI 모델로 면역원성 예측 → 상위 후보 선정
3. 선정된 신항원으로 맞춤형 암 백신 설계
4. 의료진에게 백신 제조 가이드 제공

💡 [기대 효과]
• 환자별 최적화된 암 백신 설계
• 치료 효과 향상 및 부작용 최소화
• 폐암 외 다른 암종으로 확장 가능
• 정밀 의료 시대의 핵심 기술

📁 [생성된 파일]
✅ lung_cancer_model.keras (학습된 모델)
✅ lung_cancer_FINAL_report.html (상세 데이터 리포트)
✅ eda_analysis_final.png (화려한 시각화)
✅ training_results.png (학습 과정)
✅ evaluation_results.png (성능 평가)

{"="*70}
프로젝트 완료! 🎉
{"="*70}
""")